# QTAlgo Super26 Strategy Analysis

This notebook demonstrates how to use the walk-forward optimization framework for the Super26 strategy.

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import yaml
from datetime import datetime

from src.data.loader import DataLoader
from src.strategy.indicators import calculate_all_indicators
from src.strategy.signals import generate_all_signals
from src.optimization.parameter_space import ParameterSpace
from src.utils.plotting import plot_equity_curve, plot_signal_distribution

## Load Data

In [ ]:
# Load data for SPY
loader = DataLoader(source='yfinance')
df = loader.load_data('SPY', start_date='2020-01-01', end_date='2023-12-31', timeframe='1d')

print(f"Loaded {len(df)} bars")
df.head()

## Load Strategy Parameters

In [ ]:
# Load default parameters
with open('../config/strategy_params.yaml', 'r') as f:
    config = yaml.safe_load(f)

# Flatten config
params = {}
for section, values in config.items():
    if isinstance(values, dict):
        params.update(values)

print("Strategy Parameters:")
for key, value in params.items():
    print(f"  {key}: {value}")

## Calculate Indicators

In [ ]:
# Calculate all indicators
df_with_indicators = calculate_all_indicators(df.copy(), params)

print("\nIndicators calculated:")
print(df_with_indicators.columns.tolist())

# Display last few rows
df_with_indicators.tail()

## Generate Trading Signals

In [ ]:
# Generate signals
df_with_signals = generate_all_signals(df_with_indicators, params)

print("\nSignal columns:")
signal_cols = [col for col in df_with_signals.columns if 'signal' in col or 'score' in col]
print(signal_cols)

# Display signal statistics
print("\nSignal Distribution:")
print(df_with_signals['entry_signal'].value_counts())

## Visualize Signals

In [ ]:
# Plot signal distribution
fig = plot_signal_distribution(df_with_signals)
fig.show()

## Analyze Individual Indicators

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Create subplots for price and indicators
fig = make_subplots(
    rows=3, cols=1,
    subplot_titles=('Price & Signals', 'ADX', 'Total Score'),
    vertical_spacing=0.1,
    row_heights=[0.5, 0.25, 0.25]
)

# Price chart
fig.add_trace(
    go.Scatter(x=df_with_signals.index, y=df_with_signals['close'], 
               mode='lines', name='Close Price'),
    row=1, col=1
)

# Long signals
long_signals = df_with_signals[df_with_signals['entry_signal'] == 1]
fig.add_trace(
    go.Scatter(x=long_signals.index, y=long_signals['close'],
               mode='markers', name='Long Signal',
               marker=dict(color='green', size=10, symbol='triangle-up')),
    row=1, col=1
)

# Short signals
short_signals = df_with_signals[df_with_signals['entry_signal'] == -1]
fig.add_trace(
    go.Scatter(x=short_signals.index, y=short_signals['close'],
               mode='markers', name='Short Signal',
               marker=dict(color='red', size=10, symbol='triangle-down')),
    row=1, col=1
)

# ADX
fig.add_trace(
    go.Scatter(x=df_with_signals.index, y=df_with_signals['adx'],
               mode='lines', name='ADX'),
    row=2, col=1
)
fig.add_hline(y=20, line_dash="dash", line_color="red", row=2, col=1)

# Total Score
fig.add_trace(
    go.Scatter(x=df_with_signals.index, y=df_with_signals['total_score'],
               mode='lines', name='Total Score'),
    row=3, col=1
)
fig.add_hline(y=0, line_dash="dash", line_color="gray", row=3, col=1)

fig.update_layout(height=900, showlegend=True)
fig.show()

## Parameter Space Exploration

In [ ]:
# Load optimization config
with open('../config/optimization_config.yaml', 'r') as f:
    opt_config = yaml.safe_load(f)

# Create parameter space
param_space = ParameterSpace(opt_config['parameter_bounds'])

print("Parameter Space:")
for name, param_def in param_space.parameters.items():
    print(f"  {name}: [{param_def.min_value}, {param_def.max_value}]")

# Generate random samples
samples = param_space.sample_random(n_samples=5)
print("\nRandom Parameter Samples:")
for i, sample in enumerate(samples, 1):
    print(f"\nSample {i}:")
    for key, value in sample.items():
        print(f"  {key}: {value}")

## Next Steps

- Run walk-forward optimization (see optimization_results.ipynb)
- Analyze parameter stability
- Compare in-sample vs out-of-sample performance
- Fine-tune strategy parameters